In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import csv
filename_raw = '/content/drive/My Drive/ggpalm_allchallenging_raw.csv'

with open(filename_raw, 'r',encoding='utf-8') as f:
  reader = csv.reader(f, delimiter=",", quotechar='"')
  data_raw = [row for row in reader]

print(len(data_raw))

115


In [3]:
filename_t5 = '/content/drive/My Drive/ggpalm_allchallenging_t5.csv'

with open(filename_t5, 'r',encoding='utf-8') as f:
  reader = csv.reader(f, delimiter=",", quotechar='"')
  data_t5 = [row for row in reader]

print(len(data_t5))

115


In [4]:
filename_condBERT = '/content/drive/My Drive/ggpalm_allchallenging_condBERT.csv'

with open(filename_condBERT, 'r',encoding='utf-8') as f:
  reader = csv.reader(f, delimiter=",", quotechar='"')
  data_condBERT = [row for row in reader]

print(len(data_condBERT))

115


In [5]:
filename_paragedi = '/content/drive/My Drive/ggpalm_allchallenging_paragedi.csv'

with open(filename_paragedi, 'r',encoding='utf-8') as f:
  reader = csv.reader(f, delimiter=",", quotechar='"')
  data_paragedi = [row for row in reader]

print(len(data_paragedi))

115


In [6]:
del data_raw[0]
print(data_raw[0])

del data_t5[0]
print(data_t5[0])

del data_condBERT[0]
print(data_condBERT[0])

del data_paragedi[0]
print(data_paragedi[0])

['Actually, to be more accurate, I see things like “OMG u', 'r so funny” OMG you are so funny', 'Actually, to be more accurate, I see things like “OMG u r so funny” OMG you are so funny', '0.067380086']
['Actually, to be more accurate, I see things like “OMG u r so funny” OMG you are so funny', '0.067380086', 'To be more precise, I witness expressions like "OMG u r so funny" and "oh my god, you are hilariously funny."', '0.044636376']
['Actually, to be more accurate, I see things like “OMG u r so funny” OMG you are so funny', '0.067380086', 'actually , to be more accurate , i see things like “ : oops u r so funny ” oops : you are so funny', '0.07450261']
['Actually, to be more accurate, I see things like “OMG u r so funny” OMG you are so funny', '0.067380086', 'actually, to be more precise, I see things like "OMG u\'re very funny.', '0.039226998']


In [7]:
text_raw = []
toxicity_raw = []
text_t5 = []
toxicity_t5 = []
text_condBERT = []
toxicity_condBERT = []
text_paragedi = []
toxicity_paragedi = []

for i in range(len(data_raw)):
  text_raw.append(data_raw[i][2])
  toxicity_raw.append(data_raw[i][3])
  text_t5.append(data_t5[i][2])
  toxicity_t5.append(data_t5[i][3])
  text_condBERT.append(data_condBERT[i][2])
  toxicity_condBERT.append(data_condBERT[i][3])
  text_paragedi.append(data_paragedi[i][2])
  toxicity_paragedi.append(data_paragedi[i][3])

In [9]:
picked_from = []
picked_text = []
picked_toxicity = []

picked_t5_count = 0
picked_condBERT_count = 0
picked_paragedi_count = 0

better_than_orig = []
toxicity_reduction = []
better_count = 0

for i in range(len(data_raw)):
  if (float(toxicity_t5[i]) < float(toxicity_condBERT[i])):
    if (float(toxicity_t5[i]) < float(toxicity_paragedi[i])):
      picked_from.append("t5")
      picked_text.append(text_t5[i])
      picked_toxicity.append(toxicity_t5[i])
      picked_t5_count += 1
    else:
      picked_from.append("paragedi")
      picked_text.append(text_paragedi[i])
      picked_toxicity.append(toxicity_paragedi[i])
      picked_paragedi_count += 1
  else:
    if (float(toxicity_condBERT[i]) < float(toxicity_paragedi[i])):
      picked_from.append("condBERT")
      picked_text.append(text_condBERT[i])
      picked_toxicity.append(toxicity_condBERT[i])
      picked_condBERT_count += 1
    else:
      picked_from.append("paragedi")
      picked_text.append(text_paragedi[i])
      picked_toxicity.append(toxicity_paragedi[i])
      picked_paragedi_count += 1

  if (picked_toxicity[i] < toxicity_raw[i]):
    better_than_orig.append(True)
    better_count += 1
  else:
    better_than_orig.append(False)

  toxicity_reduction.append(1 - float(picked_toxicity[i]) / float(toxicity_raw[i]))

In [10]:
print(picked_t5_count)
print(picked_condBERT_count)
print(picked_paragedi_count)
print(better_count)

47
22
45
106


In [11]:
filename = '/content/drive/My Drive/ggpalm_allchallenging_ensemble.csv'

schema = ['Orig_response','Orig_toxicity','T5_response','T5_toxicity','condBERT_response','condBERT_toxicity','paragedi_response','paragedi_toxicity','Ensemble_response','Ensemble_toxicity','Ensemble_from','Better','Toxicity_reduction']
with open(filename, 'w',encoding='utf-8') as f:
      writer = csv.writer(f)
      writer.writerow([g for g in schema])
      for i in range(len(data_raw)):
        writer.writerow([text_raw[i],toxicity_raw[i],text_t5[i],toxicity_t5[i],text_condBERT[i],toxicity_condBERT[i],text_paragedi[i],toxicity_paragedi[i],picked_text[i],picked_toxicity[i],picked_from[i],better_than_orig[i],toxicity_reduction[i]])
        # writer.writerows([[row] for row in answer_list])
        # writer.writerows([[row.replace("\u2019", "'").replace("\u2018", "'").replace("\u201c", "\"").replace("\u201d", "\"").replace("\u2013", "-").replace("\xe9", "e").replace("\u2026", "...")] for row in answer_list])
f.close()


In [12]:
def avg(float_list):
  sum = 0
  size = len(float_list)
  for i in range(size):
    sum += float(float_list[i])
  res = sum / size
  return res

In [13]:
res_raw = avg(toxicity_raw)
res_t5 = avg(toxicity_t5)
res_condBERT = avg(toxicity_condBERT)
res_paragedi = avg(toxicity_paragedi)
res_ensemble = avg(picked_toxicity)

print(res_raw)
print(res_t5)
print(res_condBERT)
print(res_paragedi)
print(res_ensemble)
print(1 - res_ensemble / res_raw)

0.26586463377192987
0.13954509328596487
0.13204294020175444
0.12489833647105263
0.06898020817807017
0.7405438730250811
